In [4]:
from datetime import datetime, timedelta
import json
from elasticsearch import Elasticsearch

def getdata():
    es = Elasticsearch(['http://10.251.151.76:9200'])
    
    # Specify the time range for data selection
    start_time = datetime.now() - timedelta(minutes=5)
    end_time = datetime.now()
    
    # Dynamically construct the index name based on the current date
    index_date = end_time.strftime("%Y.%m.%d")
    index_name = f"logstash-test-{index_date}"
    
    scroll_size = 10000
    search_body = {
        "query": {
            "bool": {
                "must": [
                    {"range": {"@timestamp": {"gte": start_time, "lte": end_time}}},  # Filter by timestamp
                ]
            }
        },
        "size": scroll_size,
    }

    response = es.search(index=index_name, body=search_body, scroll='100m')
    scroll_id = response['_scroll_id']
    results = []

    while True:
        hits = response['hits']['hits']
        if not hits:
            break
        results.extend([hit['_source'] for hit in hits])
        response = es.scroll(scroll_id=scroll_id, scroll='100m')
    
    # Convert the results to JSON format
    json_results = json.dumps(results, indent=2)
    
    # Save the results to a file named 'data.json'
    with open('data.json', 'w') as f:
        f.write(json_results)

getdata()
import json

segments = {}
# ฟังก์ชันสำหรับ segment packets และแสดง feature ของแต่ละ segment
def segment_packets(packets):
    global segments  # เก็บ segment แต่ละอันพร้อม feature
    for packet in packets:
        src_ip = packet.get('srcip', None)
        dst_ip = packet.get('dstip', None)
        src_port = packet.get('srcport', None)
        dst_port = packet.get('dstport', None)
        sentbyte = int(packet.get('sentbyte', 0))  # อ่านค่า sbytes และแปลงเป็นจำนวนเต็ม
        rcvdbyte = int(packet.get('rcvdbyte', 0))  # อ่านค่า dbytes และแปลงเป็นจำนวนเต็ม
        sentpkt = int(packet.get('sentpkt', 0))  # อ่านค่า spkts และแปลงเป็นจำนวนเต็ม
        rcvdpkt = int(packet.get('rcvdpkt', 0))  # อ่านค่า dpkts และแปลงเป็นจำนวนเต็ม
        service = packet.get('service', None)
        timestamp = packet.get('timestamp', None)  # เพิ่มการดึงค่า timestamp
        duration = packet.get('duration', None)

        segment_key = (src_ip, src_port, dst_ip, dst_port)  # กำหนด segment_key ใหม่


        # คำนวณค่า dinpkt และ sinpkt สำหรับแต่ละ packet
        if sentpkt > 1:
            duration = int(duration)
            dinpkt = duration / (sentpkt - 1)
            packet['dinpkt'] = dinpkt
        if rcvdpkt > 1:
            duration = int(duration)
            sinpkt = duration / (rcvdpkt - 1)
            packet['sinpkt'] = sinpkt

        if segment_key not in segments:
            segments[segment_key] = {'packets': [], 'ct_srv_src': 0, 'is_sm_ips_ports': 0, 'is_ftp_login': 0, 'ct_srv_dst': 0, 'ct_dst_ltm': 0, 'ct_src_ltm': 0, 'ct_src_dport_ltm': 0, 'ct_dst_sport_ltm': 0 ,'ct_dst_src_ltm': 0,'last_timestamp': None, 'sbytes': 0, 'dbytes': 0, 'spkts': 0, 'dpkts': 0,'response_body_len': 0 }  # เพิ่ม last_timestamp, sbytes, dbytes, spkts, และ dpkts ใน segment
        segments[segment_key]['packets'].append(packet)  # เพิ่มข้อมูลเรียบร้อย
        
        segments[segment_key]['sbytes'] += sentbyte  # เพิ่มขนาดของข้อมูลที่ถูกส่งออก
        segments[segment_key]['dbytes'] += rcvdbyte  # เพิ่มขนาดของข้อมูลที่ถูกรับเข้า
        segments[segment_key]['spkts'] += sentpkt  # เพิ่มจำนวนของแพ็กเก็ตที่ถูกส่งออก
        segments[segment_key]['dpkts'] += rcvdpkt  # เพิ่มจำนวนของแพ็กเก็ตที่ถูกรับเข้า

        segments[segment_key]['packets'].append(packet)  # เพิ่มข้อมูลเรียบร้อย
        if timestamp and (not segments[segment_key]['last_timestamp'] or timestamp > segments[segment_key]['last_timestamp']):
            segments[segment_key]['last_timestamp'] = timestamp  # กำหนด last_timestamp เป็น timestamp ล่าสุด

    # คำนวณค่า ct_srv_src, is_sm_ips_ports, is_ftp_login, ct_srv_dst, ct_dst_ltm และ ct_src_ltm สำหรับแต่ละ segment
    for segment, data in segments.items():
        # คำนวณ ct_srv_src
        service_count_src = {}
        for packet in data['packets']:
            src_ip = packet.get('srcip', None)
            service_key = packet.get('service', None)
            if service_key and src_ip:
                connection_key = (src_ip, service_key)
                if connection_key in service_count_src:
                    service_count_src[connection_key] += 1
                else:
                    service_count_src[connection_key] = 1
        max_count_src = max(service_count_src.values()) if service_count_src else 0
        data['ct_srv_src'] = max_count_src if max_count_src <= 100 else 100  # ระบุค่าเท่ากับจำนวนการเชื่อมต่อสูงสุด หรือ 100 ตามเงื่อนไข

        # คำนวณ is_sm_ips_ports
        is_sm_ips_ports = 1 if segment[0] == segment[2] and segment[1] == segment[3] else 0
        data['is_sm_ips_ports'] = is_sm_ips_ports
        
        # คำนวณ is_ftp_login
        for packet in data['packets']:
            if 'ftp' in packet.get('service', '').lower():
                if 'user' in packet and 'pass' in packet:
                    data['is_ftp_login'] = 1
                    break

        # คำนวณ ct_srv_dst
        service_count_dst = {}
        for packet in data['packets']:
            dst_ip = packet.get('dstip', None)
            service_key = packet.get('service', None)
            if service_key and dst_ip:
                connection_key = (dst_ip, service_key)
                if connection_key in service_count_dst:
                    service_count_dst[connection_key] += 1
                else:
                    service_count_dst[connection_key] = 1
        max_count_dst = max(service_count_dst.values()) if service_count_dst else 0
        data['ct_srv_dst'] = max_count_dst

        # คำนวณ ct_dst_ltm
        dst_count = {}
        for packet in data['packets']:
            dst_ip = packet.get('dstip', None)
            if dst_ip:
                if dst_ip in dst_count:
                    dst_count[dst_ip] += 1
                else:
                    dst_count[dst_ip] = 1
        data['ct_dst_ltm'] = len(dst_count)

        # คำนวณ ct_src_ltm
        src_count = {}
        for packet in data['packets']:
            src_ip = packet.get('srcip', None)
            if src_ip:
                if src_ip in src_count:
                    src_count[src_ip] += 1
                else:
                    src_count[src_ip] = 1
        data['ct_src_ltm'] = len(src_count)

        # คำนวณ ct_src_dport_ltm
        src_dport_count = {}
        for packet in data['packets']:
            src_ip = packet.get('srcip', None)
            dst_port = packet.get('dstport', None)
            if src_ip and dst_port:
                connection_key = (src_ip, dst_port)
                if connection_key in src_dport_count:
                    src_dport_count[connection_key] += 1
                else:
                    src_dport_count[connection_key] = 1

        ct_src_dport_ltm = sum(1 for count in src_dport_count.values() if count <= 100)
        data['ct_src_dport_ltm'] = ct_src_dport_ltm

        # คำนวณ ct_dst_sport_ltm
        dst_sport_count = {}
        for packet in data['packets']:
            dst_ip = packet.get('dstip', None)
            src_port = packet.get('srcport', None)
            if dst_ip and src_port:
                connection_key = (dst_ip, src_port)
                if connection_key in dst_sport_count:
                    dst_sport_count[connection_key] += 1
                else:
                    dst_sport_count[connection_key] = 1

        ct_dst_sport_ltm = sum(1 for count in dst_sport_count.values() if count <= 100)
        data['ct_dst_sport_ltm'] = ct_dst_sport_ltm

        # คำนวณ ct_dst_src_ltm
        dst_src_count = {}
        for packet in data['packets']:
            src_ip = packet.get('srcip', None)
            dst_ip = packet.get('dstip', None)
            if src_ip and dst_ip:
                connection_key = (src_ip, dst_ip)
                if connection_key in dst_src_count:
                    dst_src_count[connection_key] += 1
                else:
                    dst_src_count[connection_key] = 1

        ct_dst_src_ltm = sum(1 for count in dst_src_count.values() if count <= 100)
        data['ct_dst_src_ltm'] = ct_dst_src_ltm

        # คำนวณฟีเจอร์ response_body_len
        response_body_len = int(packet.get('rcvdbyte', 0)) - int(packet.get('sentbyte', 0))
        data['response_body_len'] = response_body_len

    # แก้ไขส่วนการ segment_packets()
         
    for segment, data in segments.items():
        total_sbytes = 0  # เพิ่มตัวแปรสำหรับเก็บค่ารวมของ sbytes
        total_dbytes = 0  # เพิ่มตัวแปรสำหรับเก็บค่ารวมของ dbytes
        total_spkts = 0   # เพิ่มตัวแปรสำหรับเก็บค่ารวมของ spkts
        total_dpkts = 0   # เพิ่มตัวแปรสำหรับเก็บค่ารวมของ dpkts
        
        for packet in data['packets']:
            total_sbytes += int(packet.get('sentbyte', 0))
            total_dbytes += int(packet.get('rcvdbyte', 0))
            total_spkts += int(packet.get('sentpkt', 0))
            total_dpkts += int(packet.get('rcvdpkt', 0))
        
        data['sbytes'] = total_sbytes
        data['dbytes'] = total_dbytes
        data['spkts'] = total_spkts
        data['dpkts'] = total_dpkts
     # เพิ่ม total_packets ด้วยจำนวนของ spkts และ dpkts
        total_packets = data['spkts'] + data['dpkts']
        data['total_packets'] = total_packets
    
    # # แสดง feature ของแต่ละ segment
    # for segment, data in segments.items():
    #     print("Segment:", segment)
    #     print("dinpkt", data['packets'][0].get('dinpkt', '0'))  # แสดงค่า dinpkt ของ packet แรกใน segment หรือ N/A หากไม่มีค่า
    #     print("sinpkt", data['packets'][0].get('sinpkt', '0'))
    #     print("total_packets:", data['total_packets'])
    #     print("sbytes", data['sbytes'])  
    #     print("dbytes", data['dbytes'])  
    #     print("spkts", data['spkts'])    
    #     print("dpkts", data['dpkts'])
    #     print("ct_srv_src:", data['ct_srv_src'])
    #     print("is_sm_ips_ports:", data['is_sm_ips_ports'])
    #     print("is_ftp_login:", data['is_ftp_login'])
    #     print("ct_srv_dst:", data['ct_srv_dst'])
    #     print("ct_dst_ltm:", data['ct_dst_ltm'])
    #     print("ct_src_ltm:", data['ct_src_ltm'])
    #     print("ct_src_dport_ltm:", data['ct_src_dport_ltm'])
    #     print("ct_dst_sport_ltm:", data['ct_dst_sport_ltm'])
    #     print("ct_dst_src_ltm:", data['ct_dst_src_ltm'])
    #     print("response_body_len:", data['response_body_len'])
    #     # print("Packets:")
    #     # for packet in data['packets']:
    #     #     print(json.dumps(packet, indent=4))  # แสดงข้อมูลในรูปแบบ JSON
    #     print("-" * 50)
    # # คืนค่า dictionary ของ segments
    return segments

# ฟังก์ชันหลักสำหรับอ่านข้อมูลจากไฟล์ JSON และสร้าง packets
def main(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    
    packets = []  # เก็บ packets ที่อ่านจากไฟล์ JSON
    for entry in data:
        packets.append(entry if entry else None)  #

    segments = segment_packets(packets)

    # นับและแสดงจำนวน segment ทั้งหมด
    print("Total segments:", len(segments))

if __name__ == "__main__":
    filename = "data.json"  # ชื่อไฟล์ JSON ที่ต้องการใช้
    main(filename)

    import csv

def save_segments_to_csv(segments, filename):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['dinpkt', 'sinpkt', 'sbytes', 'dbytes', 'spkts', 'dpkts', 'ct_srv_src', 'is_sm_ips_ports', 'is_ftp_login', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'response_body_len']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for segment, data in segments.items():
            writer.writerow({
                'dinpkt': data['packets'][0].get('dinpkt', '0'),
                'sinpkt': data['packets'][0].get('sinpkt', '0'),
                'sbytes': data['sbytes'],
                'dbytes': data['dbytes'],
                'spkts': data['spkts'],
                'dpkts': data['dpkts'],
                'ct_srv_src': data['ct_srv_src'],
                'is_sm_ips_ports': data['is_sm_ips_ports'],
                'is_ftp_login': data['is_ftp_login'],
                'ct_srv_dst': data['ct_srv_dst'],
                'ct_dst_ltm': data['ct_dst_ltm'],
                'ct_src_ltm': data['ct_src_ltm'],
                'ct_src_dport_ltm': data['ct_src_dport_ltm'],
                'ct_dst_sport_ltm': data['ct_dst_sport_ltm'],
                'ct_dst_src_ltm': data['ct_dst_src_ltm'],
                'response_body_len': data['response_body_len']
            })

# เรียกใช้ฟังก์ชันเพื่อเซฟข้อมูลลงในไฟล์ CSV
save_segments_to_csv(segments, 'segments.csv')


Total segments: 41060


In [5]:
import time
import pandas as pd
import joblib



def orchestrate_execution():
    while True:
        # Run Elasticsearch data retrieval
        getdata()  
        
        # Process the retrieved data
        main("data.json")  
        
        # Save processed data to CSV
        save_segments_to_csv(segments, 'segments.csv')  

        # Load model and preprocessor
        model_path = 'rf_classifier+FS+DS2.pkl'
        preprocessor_path = 'preprocessor+FS+DS2.pkl'
        rf_model = joblib.load(model_path)
        preprocessor = joblib.load(preprocessor_path)

        # Load and preprocess new data
        new_data_path = r"segments.csv"
        new_data = pd.read_csv(new_data_path)
        new_features = ['sbytes', 'dbytes', 'spkts', 'dpkts',
                        'response_body_len', 'sinpkt', 'dinpkt', 'is_sm_ips_ports', 
                        'is_ftp_login', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 
                        'ct_src_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm']
        X_test_new = new_data[new_features]
        X_new = preprocessor.transform(X_test_new)

        # Predict probabilities
        predictions_proba = rf_model.predict_proba(X_new)

        # Create DataFrame from probabilities
        probabilities_df = pd.DataFrame(predictions_proba, columns=['Probability_Normal', 'Probability_Attack'])

        # Display predictions
        print(probabilities_df)

        time.sleep(300)

if __name__ == "__main__":
    orchestrate_execution()


Total segments: 41897
       Probability_Normal  Probability_Attack
0                1.000000            0.000000
1                0.940000            0.060000
2                0.120044            0.879956
3                1.000000            0.000000
4                0.649980            0.350020
...                   ...                 ...
41892            0.590359            0.409641
41893            0.590359            0.409641
41894            0.590359            0.409641
41895            0.590359            0.409641
41896            0.372509            0.627491

[41897 rows x 2 columns]
Total segments: 52815
       Probability_Normal  Probability_Attack
0                1.000000            0.000000
1                0.920000            0.080000
2                0.120044            0.879956
3                1.000000            0.000000
4                0.649980            0.350020
...                   ...                 ...
52810            0.590359            0.409641
52811     